In [49]:
import pandas as pd
import numpy as np

In [50]:
#loading latest (07/29/16) metadata table- 11297 subjects currently
ag_md= pd.read_table("../data/agp_md_072916_raw.txt", dtype=str, index_col="#SampleID") #importing metadata file as pandas
ag_md.shape

(11297, 462)

In [51]:
#loading the 39 agp samples that are a part of mm but were not included in agp metadata *initially* because they weren't sequenced
ag_missing_md= pd.read_table("../data/ag_missing_md.txt", sep='\t', dtype=str) #importing metadata file as pandas
ag_missing_md.shape

(39, 168)

In [52]:
l1=(list(ag_md.columns.values)) 
l2=(list(ag_missing_md.columns.values))

In [53]:
#finding non-overlapping columns
c=0
for k in l1:
    if k in l2:
        c=c+1
print c

164


NOTE: 
A. 164 columns in the two data sets are common
B. ag_md has 298 non-overlapping columns (VIOSCREEN etc.)
C. ag_missing_md has the following non-overlapping columns:

- sample_name -> change it into #SampleID by concatenating "10317." etc. to the front
- DESCRIPTION (American Gut Project Stool sample. Add the column to ag_md)
- SCIENTIFIC_NAME (ag_missing_md has scientific_name; ag_md has host_common_name) -> delete scientific name from missing, add common name in missing
- TAXON_ID (ag_missing has 408170 i.e human gut metagenome; ag_md has nothing corresponding this)

In [54]:
#appending prefix to the sample_name
ag_missing_md["sample_name"]="10317.0000"+ag_missing_md["sample_name"] 

In [55]:
#renaming sample_name to #SampleID to make it consistent with ag_1k
ag_missing_md=ag_missing_md.rename(columns={"sample_name":"#SampleID"}) 

In [56]:
#indexing ag_missing by #SampleID
ag_missing_md=ag_missing_md.set_index("#SampleID")

In [57]:
#convert column names of both the pandas md files to lower case
ag_md.columns = map(str.lower, ag_md.columns)
ag_missing_md.columns = map(str.lower, ag_missing_md.columns)

In [58]:
#add Description to ag_md
ag_md['description'] = "American Gut Project Stool sample"

In [59]:
#remove SCIENTIFIC_NAME column in ag_missing. Add host_common_name
del ag_missing_md['scientific_name']
ag_missing_md['host_common_name']="human"

In [60]:
#add TAXON_ID to ag_md
ag_md['taxon_id']="408170"

In [61]:
missing=[]
included=[]

for idx in ag_missing_md.index:
    if idx not in ag_md.index:
        missing.append(idx)
    else:
        included.append(idx)

print("The number of MM samples still missing in AGP metadata file are: %d") %(len(missing))
print("The number of MM samples that have been included in the latest AGP metadata file used are: %d") %(len(included))
#30 samples in ag_missing_md have been included in the latest ag_md (as of july 29, 2016)

The number of MM samples still missing in AGP metadata file are: 9
The number of MM samples that have been included in the latest AGP metadata file used are: 30


In [62]:
missing # number of samples which are still not sequenced through AGP but belong to the MM cohort

['10317.000036073',
 '10317.000036074',
 '10317.000036079',
 '10317.000036385',
 '10317.000036403',
 '10317.000036431',
 '10317.000036922',
 '10317.000036930',
 '10317.000036948']

In [63]:
#non-overlapping columns
nonoverlap=[]
l1=(list(ag_md.columns.values))
l2=(list(ag_missing_md.columns.values))
for col in l2:
    if col not in l1:
        nonpverlap.append(col)
if col==[]:
    print("All the columns on ag_missing_md are included in ag_md")

All the columns in ag_missing_md are in ag_md. (ag_md has extra columns not included in ag_missing_md)

In [64]:
col=list(ag_md.columns)

In [65]:
len(col)

463

In [66]:
notlogged=pd.DataFrame(index=['10317.000030770','10317.000030773','10317.000030776','10317.000030777','10317.000030778','10317.000030766','10317.000030768','10317.000030781','10317.000030780'], columns=col)

In [67]:
#dropping samples that have been included in the latest agp run from the table with missing samples 
ag_missing_md=ag_missing_md.drop(included)
ag_missing_md.shape

(9, 166)

In [68]:
#appending not logged samples to the list with missing samples
mm_missing_all=ag_missing_md.append(notlogged)

9 samples were not logged due to consent issues. And 9 have not been sequences through AGP. So, we dont have data for a total of 18 samples in the MM cohort. 

In [70]:
#saving a list of missing samples
mm_missing_all.to_csv("../data/mm_missing_all.txt",sep='\t', float_format='string')

Adding these 18 samples to a master metadata file, to keep everything at one place. 

In [71]:
ag_merged_md=ag_md.append(mm_missing_all)

In [72]:
ag_merged_md.shape #11297 (orignally in agp table) + 18  missing samples

(11315, 463)

In [73]:
ag_merged_md.to_csv("../data/ag_all_plus_mm_missing.txt", sep='\t', float_format='string')